In [5]:
from typing import TypedDict
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [7]:
model = ChatOpenAI(model = 'gpt-40-mini')

In [8]:
class EvaluationSchema(BaseModel):
    feedback: str= Field(description="Detailed feedback for the easy")
    feedback: int= Field(description="Score out of 10", ge=0, le=10)

In [9]:
structured_model = model.with_structured_output(EvaluationSchema)